In [4]:
import numpy as np
import xarray as xr
import os
import pandas as pd
import glob

In [5]:
import sys ; sys.path.append("../..")
from ppe_tools import *

In [6]:
# set directories
exp_dir = '/glade/work/linnia/CLM-PPE-LAI_tests/exp1_EmBE/'

### 1. Define Parameters and Ranges

In [7]:
csv='lhc_220926.csv'

In [4]:
#data_url = '"https://docs.google.com/spreadsheets/d/e/2PACX-1vQLjMUT7sgunI4XkbGWq_RrjrLhR8-N0RiojM7h3lbSsN5-jgKIFyiJtPJeQkTEf2TL4ouLu_dqmEkg/pub?gid=737792973&single=true&output=csv"'
#cmd = 'curl -L '+data_url+' > '+csv # need to add -L option to force redirects
#os.system(cmd)

In [8]:
df = pd.read_csv(csv)
params=df['param'].values
params

array(['FUN_fracfixers', 'KCN', 'a_fix', 'crit_dayl', 'd_max', 'fff',
       'froot_leaf', 'fstor2tran', 'grperc', 'jmaxb0', 'jmaxb1', 'kcha',
       'kmax', 'krmax', 'leaf_long', 'leafcn', 'lmr_intercept_atkin',
       'lmrha', 'lmrhd', 'medlynintercept', 'medlynslope', 'nstem',
       'psi50', 'q10_mr', 'slatop', 'soilpsi_off', 'stem_leaf',
       'sucsat_sf', 'theta_cj', 'tpu25ratio', 'tpuse_sf', 'wc2wjb0'],
      dtype=object)

In [9]:
kcns=['kc_nonmyc',
        'kn_nonmyc',
        'akc_active',
        'akn_active',
        'ekc_active',
        'ekn_active']
kcns

['kc_nonmyc',
 'kn_nonmyc',
 'akc_active',
 'akn_active',
 'ekc_active',
 'ekn_active']

### 2. Build Parameter Dictionary

In [10]:
### we should maybe make this a function?
lhcs = {}
for p in params:
    ix       = df['param']==p
    minval   = df['min'][ix].values[0]
    maxval   = df['max'][ix].values[0]
    pftmin   = df['pft_mins'][ix].values[0]
    pftmax   = df['pft_maxs'][ix].values[0]
    thisloc  = df['loc'][ix].values[0]
    if p=='KCN':
        flag='KCN'
    else:
        flag=''
    
    needs_pft = (minval=='pft')
    if needs_pft:
        thismin = np.fromstring(pftmin, dtype='float', sep=',')
    elif 'percent' in minval:
        thismin = minval
    else:
        thismin = np.array(float(minval))

    needs_pft = (maxval=='pft')
    if needs_pft:
        thismax = np.fromstring(pftmax, dtype='float', sep=',')
    elif 'percent' in maxval:
        thismax = maxval
    else:
        thismax = np.array(float(maxval))

    if p!='KCN':
        lhcs[p]={'min':thismin,'max':thismax,'loc':thisloc,'flagged':[]}
    else:
        flagged={}
        for kcn in kcns:
            flagged[kcn]={'min':thismin,'max':thismax,'loc':thisloc}
        lhcs[p]={'min':[],'max':[],'loc':thisloc,'flagged':flagged}

### 3. Define Ensemble

##### a. If you would like to generate a new Latin-Hypercube sample, skip to step 4
##### b. If you already know the parameter sets to run, continue on

In [11]:
# read in parameter weights
# specify where your samples exist
# make sure that they are in the same order as the params in lhcs above
te = pd.read_csv('../../pyth/exp1_EmBE/exp1_EmBE_TotalError_psets.csv',index_col=0)
bm = pd.read_csv('../../pyth/exp1_EmBE/exp1_EmBE_BiomeSpecific_psets.csv',index_col=0)
psets = pd.concat([te,bm],ignore_index=True)

In [12]:
def get_p(p,i):
    return psets.iloc[i].loc[p]

In [13]:
exp1=[[get_p(p,i) for p in lhcs] for i in range(48)]

In [14]:
#Rows are ensemble members and columns are parameters (scale factors)
np.shape(exp1)

(48, 32)

### 4. Create the Ensemble Object

In [36]:
# 4a. instantiate the Ensemble object
basefile = '/glade/campaign/asp/djk2120/PPEn11/paramfiles/OAAT0000.nc'
pdir = exp_dir + 'paramfiles/'
ndir = exp_dir + 'namelist_mods/'
x    = Ensemble(basefile,pdir,ndir)

# 4b. add the new ensemble members
prefix = 'exp1_EmBE'   #make your own prefix
nextnum=1
n_samples=48  # set the number of ensemble members 
x.add_lhcs(lhcs,prefix,nextnum,n_samples,lhc=exp1) # for new LHC sample set lhc=None

### 5. Write the param_files

In [29]:
paramset_file = exp_dir + 'psets_exp1_EBE_230419.txt'
x.write(lhcfile=paramset_file)

In [41]:
%%bash -s "$paramset_file"
# Check that the file was created
paramset_file=$1 

stat $paramset_file

  File: ‘/glade/work/linnia/CLM-PPE-LAI_tests/exp1_EmBE/psets_exp1_EBE_230419.txt’
  Size: 29954     	Blocks: 64         IO Block: 8388608 regular file
Device: 2ch/44d	Inode: 5596486690  Links: 1
Access: (0644/-rw-r--r--)  Uid: (35240/  linnia)   Gid: ( 1000/    ncar)
Access: 2023-04-19 14:51:40.493541000 -0600
Modify: 2023-04-19 14:51:40.497087000 -0600
Change: 2023-04-19 14:51:40.496293794 -0600
 Birth: -
